# Tutorial: Learn how to use TabularDatasets in Azure Machine Learning

In this tutorial, you will learn how to use Azure Machine Learning Datasets to train a classification model with the Azure Machine Learning SDK for Python. You will:

&#x2611; Setup a Python environment and import packages

&#x2611; Load the Titanic data from your Azure Blob Storage. (The [original data](https://www.kaggle.com/c/titanic/data) can be found on Kaggle)

&#x2611; Create and register a TabularDataset in your workspace

&#x2611; Train a classification model using the TabularDataset

## Pre-requisites:
To create and work with datasets, you need:
* An Azure subscription. If you donâ€™t have an Azure subscription, create a free account before you begin. Try the [free or paid version of Azure Machine Learning service](https://aka.ms/AMLFree) today.
* An [Azure Machine Learning service workspace](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-manage-workspace)
* The [Azure Machine Learning SDK for Python installed](https://docs.microsoft.com/python/api/overview/azure/ml/install?view=azure-ml-py), which includes the azureml-datasets package.

Data and train.py script to store in your Azure Blob Storage Account.
 * [Titanic data](./train-dataset/Titanic.csv)
 * [train.py](./train-dataset/train.py)

## Initialize a Workspace

Initialize a workspace object from persisted configuration

In [ ]:
import azureml.core
from azureml.core import Workspace, Datastore, Dataset

# Get existing workspace from config.json file in the same folder as the tutorial notebook
# You can download the config file from your workspace
workspace = Workspace.from_config()
print(workspace)

## Create a TabularDataset

Datasets are categorized into various types based on how users consume them in training. In this tutorial, you will create and use a TabularDataset in training. A TabularDataset represents data in a tabular format by parsing the provided file or list of files. TabularDataset can be created from csv, tsv, parquet files, SQL query results etc. For the complete list, please visit our [documentation](https://aka.ms/tabulardataset-api-reference). It provides you with the ability to materialize the data into a pandas DataFrame.

By creating a dataset, you create a reference to the data source location, along with a copy of its metadata. The data remains in  its existing location, so no extra storage cost is incurred.

We will now upload the [Titanic data](./train-dataset/Titanic.csv) to the default datastore(blob) within your workspace..

In [ ]:
datastore = workspace.get_default_datastore()
datastore.upload_files(files = ['./train-dataset/Titanic.csv'],
                       target_path = 'train-dataset/',
                       overwrite = True,
                       show_progress = True)

Then we will create an unregistered TabularDataset pointing to the path in the datastore. We also support create a Dataset from multiple paths. [learn more](https://aka.ms/azureml/howto/createdatasets) 

In [ ]:
dataset = Dataset.Tabular.from_delimited_files(path = [(datastore, 'train-dataset/Titanic.csv')])

#preview the first 3 rows of the dataset
dataset.take(3).to_pandas_dataframe()

Use the `register()` method to register datasets to your workspace so they can be shared with others, reused across various experiments, and refered to by name in your training script.

In [ ]:
dataset = dataset.register(workspace = workspace,
                           name = 'titanic dataset',
                           description='training dataset')

## Create or Attach existing AmlCompute
You will need to create a [compute target](https://docs.microsoft.com/azure/machine-learning/service/concept-azure-machine-learning-architecture#compute-target) for your training. In this tutorial, you create `AmlCompute` as your training compute resource.

**Creation of AmlCompute takes approximately 5 minutes.** If the AmlCompute with that name is already in your workspace this code will skip the creation process.

As with other Azure services, there are limits on certain resources (e.g. AmlCompute) associated with the Azure Machine Learning service. Please read [this article](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-manage-quotas) on the default limits and how to request more quota.

In [ ]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget

# Choose a name for your cluster.
amlcompute_cluster_name = "your cluster name"

found = False
# Check if this compute target already exists in the workspace.
cts = workspace.compute_targets
if amlcompute_cluster_name in cts and cts[amlcompute_cluster_name].type == 'AmlCompute':
    found = True
    print('Found existing compute target.')
    compute_target = cts[amlcompute_cluster_name]

if not found:
    print('Creating a new compute target...')
    provisioning_config = AmlCompute.provisioning_configuration(vm_size = "STANDARD_D2_V2", # for GPU, use "STANDARD_NC6"
                                                                #vm_priority = 'lowpriority', # optional
                                                                max_nodes = 6)

    # Create the cluster.\n",
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, provisioning_config)

print('Checking cluster status...')
# Can poll for a minimum number of nodes and for a specific timeout.
# If no min_node_count is provided, it will use the scale settings for the cluster.
compute_target.wait_for_completion(show_output = True, min_node_count = None, timeout_in_minutes = 20)

# For a more detailed view of current AmlCompute status, use get_status().

## Create an Experiment
**Experiment** is a logical container in an Azure ML Workspace. It hosts run records which can include run metrics and output artifacts from your experiments.

In [ ]:
from azureml.core import Experiment

experiment_name = 'training-datasets'
experiment = Experiment(workspace = workspace, name = experiment_name)
project_folder = './train-dataset/'

## Configure & Run

In [ ]:
from azureml.core.runconfig import RunConfiguration
from azureml.core.conda_dependencies import CondaDependencies
import pkg_resources

# create a new RunConfig object
conda_run_config = RunConfiguration(framework="python")

# Set compute target to AmlCompute
conda_run_config.target = compute_target
conda_run_config.environment.docker.enabled = True
conda_run_config.environment.docker.base_image = azureml.core.runconfig.DEFAULT_CPU_IMAGE

dprep_dependency = 'azureml-dataprep==' + pkg_resources.get_distribution("azureml-dataprep").version

cd = CondaDependencies.create(pip_packages=['azureml-sdk', 'scikit-learn', 'pandas', dprep_dependency])
conda_run_config.environment.python.conda_dependencies = cd

In [ ]:
# create a new RunConfig object
run_config = RunConfiguration()

run_config.environment.python.user_managed_dependencies = True

from azureml.core import Run
from azureml.core import ScriptRunConfig

src = ScriptRunConfig(source_directory=project_folder, 
                      script='train.py', 
                      run_config=conda_run_config) 
run = experiment.submit(config=src)
run.wait_for_completion(show_output=True)

## View run history details

In [ ]:
run

You have now finished using a dataset from start to finish of your experiment!

![Impressions](https://PixelServer20190423114238.azurewebsites.net/api/impressions/MachineLearningNotebooks/work-with-data/datasets/datasets-tutorial/datasets-tutorial.png)